# Smoothing and Mapping

本章我们介绍平滑与建图算法(Smoothing and Mapping, SAM)算法，与SLAM不同的是，它没有增量式的技巧，是一种先采集数据然后进行后处理的算法。即在获得所有观测Z后，对所有状态变量X求最大后验概率$\mathbb{P}(X|Z)$，本章将介绍求MAP的优化算法。

## 一、SLAM中的因子图
真实SLAM任务中的因子图如下图所示，它由连接相邻位姿变量节点的里程计因子(odometry factor)和连接位姿变量节点和路标变量节点的似然因子组成。
<div align="center"><img src="../resources/chapter2/slam_factor_graph.png" width=55%></div>

除了先验因子之外，其余SLAM问题的所有因子都是非线性的，后面将介绍如何求解后验概率全局最大值(本质就是对非线性目标函数不断的线性化，求解大规模稀疏线性系统)。

## 二、对非线性因子图的MAP Inference
由于因子一般都是Gaussian函数，多个因子相乘取Log后，则可以变成一个非线性最小二乘求和形式的目标函数。
$$\begin{equation}
X^{MAP} = argmin_X \sum_i||h_i(X_i) - z_i||_{\Sigma_i}^2
\end{equation}$$
### 2.1 残差函数线性化
线性化即对目标函数在局部进行一阶Taylor展开。令残差$r_i(X_i) = h_i(X_i) - z_i$
$$r_i(X_i) = r_i(X_i^0 + \Delta_i) \approx r_i(X_i^0) + J_i\Delta_i$$

这时我们可以得到一个线性最小二乘问题。
$$\Delta^* = argmin_\Delta \sum_i ||r_i(X_i^0) + J_i\Delta_i||_{\Sigma_i}^2 \\ 
        = argmin_\Delta \sum_i ||\Sigma_i^{-1/2}J_i\Delta_i + \Sigma_i^{-1/2}r_i(X_i^0)||_{2}^2
$$
令$A_i = \Sigma_i^{-1/2}J_i$,$b_i = -\Sigma_i^{-1/2}r_i(X_i)$，则该优化问题变成最小化$||A_i\Delta_i - b_i||_2^2$的线性最小二乘问题。与$\Sigma_i^{-1/2}$相乘称为白化(whitening)，表示对残差进行加权。

**备注**：
1. 以上对一个残差进行了线性化，对应在`Ceres`库中，需要指定残差计算方法、残差函数Jacabi和信息矩阵$\Sigma_i$。
2. 不同的残差函数之间通过加号连接，因此要获得整个目标函数$$g(X) = \sum_i ||r_i(X_i)||_{\Sigma_i}^2$$的$A,b$矩阵，只须**拼接各个残差的$A_i, b_i$**，进而可以得到块稀疏Jacobian矩阵。

### 2.2 梯度下降(最速下降)
$$\Delta_{sd} = -\alpha \nabla g(X)$$
### 2.3 Gaussian-Newton
求解$||Ax - b||_2^2$最小化问题。即求解正规方程
$$A^TA\Delta_{gn} = A^Tb$$
### 2.4 LM
LM是一种结合了梯度下降和G-N法的信赖域方法，具体定义和伪代码如下所示。
<div align="center"><img src="../resources/chapter2/LM.png" width=55%></div>
<div align="center"><img src="../resources/chapter2/LM_code.png" width=55%></div>

### 2.5 Dog-Leg
略